In [1]:
import pandas as pd
import numpy as np

In [12]:
fireb = pd.read_csv('./data/hm450_cpg2manygene_firebrowse.lst', delim_whitespace=True,header=None)
fireb.columns = ['CpG','Symbol']
fireb.shape

(485577, 2)

In [45]:
def expand_cpgene_old(df) :
    s = df['Symbol'].str.split(';').apply(pd.Series,1).stack()
    i = s.index.get_level_values(0)
    df2 = df.loc[i].copy()
    df2["meth_gene"] = s.values
    df2.drop('Symbol', axis=1, inplace=True)
    return df2

def expand_cpgene(df) :
    s = df['Symbol'].str.split(';').apply(pd.Series,1).stack()
    s.index = s.index.droplevel(-1)
    df2 = df.loc[s.index].copy()
    df2["meth_gene"] = s.values
    df2.reset_index(drop=True,inplace=True)
    return df2

In [86]:
cpg2gene_1on1 = expand_cpgene(fireb)
cpg2gene_1on1.drop('Symbol', axis=1, inplace=True)
cpg2gene_1on1.head(20)

,CpG,meth_gene
0,cg00000029,RBL2
1,cg00000108,C3orf35
2,cg00000109,FNDC3B
3,cg00000236,VDAC3
4,cg00000289,ACTN1
5,cg00000292,ATP2A1
6,cg00000321,SFRP1
7,cg00000622,NIPA2
8,cg00000658,MAN1B1
9,cg00000714,TSEN34


In [81]:
raw = pd.read_csv('./data/hm450_tcga_cpg_descriptions.txt', delim_whitespace=True)
raw.columns = ['CpG',u'Chromosome', u'Start', u'End', u'Gene_Symbol', u'Gene_Type',
       u'Transcript_ID', u'Position_to_TSS', u'CGI_Coordinate',
       u'Feature_Type']
raw = raw[['CpG', 'Gene_Symbol' , u'Chromosome', u'Start',u'End' ]]
raw.loc[:,'Meth_genes'] = raw['Gene_Symbol'].apply(lambda x: list(set(x.split(';'))))
raw = raw.drop('Gene_Symbol', axis=1)

In [84]:
cpg_location = raw.drop('Meth_genes', axis=1)

In [87]:
df = pd.merge(cpg_location, cpg2gene_1on1, how='inner')

In [121]:
df['CpG'] = df['CpG'].astype(str)
df.head()

,CpG,Chromosome,Start,End,meth_gene
0,cg00000029,chr16,53434200,53434201,RBL2
1,cg00000108,chr3,37417715,37417716,C3orf35
2,cg00000109,chr3,172198247,172198248,FNDC3B
3,cg00000236,chr8,42405776,42405777,VDAC3
4,cg00000289,chr14,68874422,68874423,ACTN1


In [144]:
#df2 = (df[(df['Chromosome'] != 'chrX') | (df['Chromosome']!='chrY') | (df['Chromosome']!='*') ])
df2 = df[df['Chromosome'].isin(['chrX','chrY','*']) == False] 

df2.to_csv('./data/cpg2manygene_description.lst', index=False)
df2[['CpG','meth_gene']].to_csv('./data/cpg2manygene.lst', index=False)

In [152]:
df.shape,df2.shape

((407090, 5), (393562, 5))

In [153]:
df2.Chromosome.value_counts()

chr1     39421
chr6     29613
chr2     26133
chr11    25396
chr17    25352
chr19    24484
chr5     23950
chr7     23843
chr3     21095
chr12    19918
chr10    18496
chr16    17914
chr4     15413
chr8     14860
chr15    12421
chr14    12103
chr20     9915
chr13     8664
chr9      8193
chr22     7897
chr18     4868
chr21     3613
Name: Chromosome, dtype: int64

In [123]:
df3 = df.groupby('meth_gene', group_keys=True).apply(lambda x: x.sort_values(['Chromosome','Start']))

In [162]:
df3.head()

CpG Chromosome     Start       End meth_gene
meth_gene                                                            
A1BG      53439   cg03123289      chr19  58347269  58347270      A1BG
          331766  cg22286978      chr19  58347440  58347441      A1BG
          170689  cg10734734      chr19  58347655  58347656      A1BG
          174399  cg11001216      chr19  58347663  58347664      A1BG
          221559  cg14222245      chr19  58350136  58350137      A1BG

In [154]:
df4 = df2.groupby('meth_gene')['CpG'].apply(list)
df5 = df4.apply(lambda x: ' ,'.join(x))

In [155]:
df4.head()

meth_gene
A1BG     [cg02957155, cg03123289, cg03630821, cg0426968...
A1CF     [cg03817621, cg10222734, cg16531903, cg2441194...
A2BP1    [cg00019838, cg00036119, cg00099017, cg0020575...
A2LD1    [cg00336946, cg01723761, cg02425595, cg0273550...
A2M      [cg00134295, cg00141925, cg00146928, cg0801204...
Name: CpG, dtype: object

In [163]:
df5.head()

meth_gene
A1BG     cg02957155 ,cg03123289 ,cg03630821 ,cg04269689...
A1CF     cg03817621 ,cg10222734 ,cg16531903 ,cg24411946...
A2BP1    cg00019838 ,cg00036119 ,cg00099017 ,cg00205756...
A2LD1    cg00336946 ,cg01723761 ,cg02425595 ,cg02735500...
A2M      cg00134295 ,cg00141925 ,cg00146928 ,cg08012045...
Name: CpG, dtype: object

In [ ]:
df5.to_csv('./data/gene2cpg_unique.lst', sep='\t', index=True)